Import `joblib` from `sklearn.externals` and `numpy` as `np`.
Also import `pandas` as `pd` and `csv`

In [ ]:
import ...

## load model

Use `joblib.load()` to load and store the model file 'stolen_model.pkl'.

In [ ]:
model = ...

## Top 10 attributes class 1

To extract the top 10 attributes leading to one class, we take advantage of our knowledge about the underlying pipeline. The pipeline has a step named `'lin_svc'` that contains the support vector machine. Use `named_steps[]` to get this step from the pipeline and call `coef_[0]` to receive the coefficients/weights from the SVM. - store the results into a variable called 'weights'. The sorting by weight has been done for you, you only have to get the top 10 features from the weights list.

In [ ]:
weights = ...
weights.toarray()
sorted_index = np.argsort(weights[0])[::-1]
top_10 = sorted_index[: ...]

Now that we have the weights we are interested in, we need to connect them to the corresponding terms. Get the named step `'tfidv'` from the pipeline and call `get_feature_names()` to store the terms as a new variable.

In [ ]:
terms = ...

for idx in top_10:
    print(terms[idx])

## Top 10 attributes class 2

Extract the top 10 attributes determining the second class (the right dimension has already been selected for you).

In [ ]:
weights = ...
weights.toarray()
sorted_index = np.argsort(weights[0])[::1]
top_10 = ..
terms = ...
for ind in top_10:
    print(terms[ind])

## lime

Create list of size 2 with the elements 'activist' and 'public' in this order and assign it to `class_names`. Create a LimeTextExplainer(), passing `class_names = class_names`. Use the `explain_instance` function (pass a text to explain as the first argument, `model.predict_proba` and `num_features=10`) and save the result to exp.

In [ ]:
# use lime to explain the prediction result of a test string
from lime import lime_text
from lime.lime_text import LimeTextExplainer

class_names = ...
explainer = ...
exp = explainer.explain_instance(... , ... , ...)
exp.as_list()

In [ ]:
# use the built-in lime visualizations
%matplotlib inline
fig = exp.as_pyplot_figure()
exp.show_in_notebook(text=True)

## eli5

**eli5** is a python package that has been built on top of lime and a couple of other explainable AI projects. Use eli5's `show_weights()` function, giving it the named step 'lin_svc' as first argument and as `vec`. Additionally define the number of features you are interested in by setting `top` to an integer of your choice. 

In [ ]:
# use eli5 to show the top N features contributing to one class
import eli5

eli5. ...

Use the eli5 function `show_prediction()` with the named step 'lin_svc' as first argument, any string as the second argument and `vec=model.named_steps['tfidv']`.

In [ ]:
# use eli5 to visualize the most important features for a sample text

## send your candidate messages to the server for evaluation
Use backdoor.py (either paste the code in here or execute it separately)

## save your candidate messages
Save your messages as a .csv using the code below.


In [ ]:
messages = [['message 1'], 
            ['message 2'], 
            ['message 3'], 
            ['message 4'], 
            ['message 5'], 
            ['message 6'], 
            ['message 7']] 
df = pd.DataFrame(messages)
df.to_csv("your_team.csv", encoding='utf-8', quoting=csv.QUOTE_ALL,header=False, index=False)